In [6]:
%pip install transformers

   ---------------------------------------- 0.0/11.6 MB ? eta -:--:--
   ---------------------------------------  11.5/11.6 MB 64.3 MB/s eta 0:00:01
   ---------------------------------------- 11.6/11.6 MB 46.8 MB/s  0:00:00

   -------------------- ------------------- 1/2 [transformers]
   -------------------- ------------------- 1/2 [transformers]
   -------------------- ------------------- 1/2 [transformers]
   -------------------- ------------------- 1/2 [transformers]
   -------------------- ------------------- 1/2 [transformers]
   -------------------- ------------------- 1/2 [transformers]
   -------------------- ------------------- 1/2 [transformers]
   -------------------- ------------------- 1/2 [transformers]
   -------------------- ------------------- 1/2 [transformers]
   -------------------- ------------------- 1/2 [transformers]
   -------------------- ------------------- 1/2 [transformers]
   -------------------- ------------------- 1/2 [transformers]
   ---------------

In [ ]:
import pandas as pd
from langchain.docstore.document import Document
from langchain.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain_community.llms import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline

# 1. 데이터 로드 및 전처리 (이전과 동일)
try:
    df = pd.read_csv('db_drug_interactions.csv')
    print("CSV 파일 로드 성공!")
    print(f"데이터 수: {len(df)}개")

    documents = []
    for _, row in df.iterrows():
        content = f"Interaction between {row['Drug 1']} and {row['Drug 2']}: {row['Interaction Description']}"
        doc = Document(
            page_content=content,
            metadata={
                'drug1': row['Drug 1'],
                'drug2': row['Drug 2']
            }
        )
        documents.append(doc)
    print("Document 객체 변환 완료!")

except FileNotFoundError:
    print("오류: 'db_drug_interactions.csv' 파일을 찾을 수 없습니다. 파일이 코드와 같은 디렉토리에 있는지 확인하세요.")
    exit()


In [ ]:


# 2. Retriever (검색기) 설정 (이전과 동일)
print("임베딩 모델 로딩 중...")
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
print("임베딩 모델 로드 완료!")

print("벡터 데이터베이스 생성 중...")
vector_store = FAISS.from_documents(documents, embeddings)
print("벡터 데이터베이스 생성 완료!")

retriever = vector_store.as_retriever()


# 3. Generator (생성기) 설정 (*** 이 부분이 변경되었습니다 ***)
# 로컬에서 실행할 언어 모델(LLM) 설정
# Hugging Face Hub API 토큰이 더 이상 필요 없습니다.
print("로컬 언어 모델 로딩 중... (최초 실행 시 모델 다운로드로 인해 시간이 오래 걸릴 수 있습니다)")

# 사용할 모델 이름 정의
model_id = "google/flan-t5-base"

# 1. Hugging Face `transformers`를 사용해 모델과 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(model_id)

# 2. `transformers`의 pipeline 생성
pipe = pipeline(
    "text2text-generation",
    model=model,
    tokenizer=tokenizer,
    max_length=128
)

# 3. LangChain의 `HuggingFacePipeline`으로 래핑(Wrapping)
llm = HuggingFacePipeline(pipeline=pipe)

print("언어 모델 설정 완료!")

# 프롬프트 템플릿 정의 (이전과 동일)
prompt_template = """
당신은 약물 상호작용 전문가입니다. 제공된 정보를 바탕으로 사용자의 질문에 대해 명확하고 간결하게 한국어로 답변해주세요.
정보를 찾을 수 없다면, "제공된 정보 내에서는 해당 내용을 찾을 수 없습니다."라고 답변하세요.

[제공된 정보]
{context}

[질문]
{question}

[답변]
"""
PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)


# 4. RetrievalQA 체인 생성 및 실행 (이전과 동일)
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    chain_type_kwargs={"prompt": PROMPT},
    return_source_documents=True
)

def ask_question(query):
    print(f"\n--- 질문: {query} ---")
    result = qa_chain.invoke(query)

    print("\n[생성된 답변]")
    print(result['result'])

    print("\n[답변 근거 문서]")
    for doc in result['source_documents']:
        print(f"- {doc.page_content}")

# --- 시스템 실행 ---
if __name__ == "__main__":
    ask_question("What is the interaction between Trioxsalen and Verteporfin?")
    ask_question("Paclitaxel과 Verteporfin의 상호작용은 무엇인가요?")

CSV 파일 로드 성공!
데이터 수: 191541개
Document 객체 변환 완료!
임베딩 모델 로딩 중...


C:\Users\mingyu\AppData\Local\Temp\ipykernel_40216\720407651.py:36: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")


ImportError: Could not import sentence_transformers python package. Please install it with `pip install sentence-transformers`.